# "Unit test" :) 

In [ ]:

DB_ULR = "neo4j://localhost:7687"
DB_USER = "neo4j"
DB_PASS = "test1234"

In [ ]:
import pandas as pd
from neo4j import GraphDatabase, RoutingControl # Python database driver

In [ ]:
driver = GraphDatabase.driver(DB_ULR, auth=(DB_USER, DB_PASS))
driver.verify_connectivity()

In [ ]:
db_name = "vector"
index_name = 'myVectorIndex'
dim = 2048
node_count = 10_000

In [ ]:
driver.execute_query(
    'create database vector if not exists options { storeFormat:"block"}',
    None,
    RoutingControl.WRITE,
    database_= 'system'
)

In [ ]:
driver.execute_query(
    'call db.index.vector.createNodeIndex($index_name, "Node", "vector", $dim, "cosine")',
    {'index_name': index_name, 'dim': dim},
    routing_= RoutingControl.WRITE,
    database_= db_name
)

In [ ]:
records, summary, keys = driver.execute_query(
    'show indexes yield * where name = $index_name return *',
    {'index_name': index_name},
    routing_= RoutingControl.READ,
    database_= db_name
)
for record in records:
    print(record)

In [ ]:
# Create nodes
with driver.session(database=db_name) as session:
    result = session.run(
        '''
        unwind range(1,$node_count) as node_id
        call {
            with node_id
            create (n:Node{id: node_id}) with n
            call db.create.setNodeVectorProperty(n, 'vector', [ x in range(1, $dim) | rand() ])
            //set n.vector = [ x in range(1, $dim) | rand() ]
        } in transactions of 5000 rows
        return count(*) as nodes_created
        ''',
        parameters={'dim': dim, 'node_count': node_count},
    ).consume()

In [ ]:
result.result_consumed_after

In [ ]:
# Add the index
driver.execute_query(
    'call db.index.vector.createNodeIndex($index_name, "Node", "vector", $dim, "cosine")',
    {'index_name': index_name, 'dim': dim},
    routing_= RoutingControl.WRITE,
    database_= db_name
)

In [ ]:
with driver.session(database=db_name) as session:
    result = session.run(
        '''
        unwind range(1,$node_count) as node_id
        call {
            with node_id
            create (n:Node{id: node_id}) with n
            with n, [ x in range(1, $dim) | rand() ] as vector
            call db.create.setNodeVectorProperty(n, 'vector', vector)
        } in transactions of 5000 rows
        return count(*) as nodes_created
        ''',
        parameters={'dim': dim, 'node_count': node_count},
    ).consume()

In [ ]:
result.counters

In [ ]:
result.result_consumed_after

In [ ]:
records, summary, keys = driver.execute_query(
    '''
    call db.index.vector.queryNodes($index_name, 1, $vector) yield node
    return node
    ''',
    vector=[1.2, 0.3],
    index_name = index_name,
    routing_= RoutingControl.READ,
    database_= db_name
)
for record in records:
    print(record)

In [11]:
records, summary, keys = driver.execute_query(
    '''
    with [ x in range(1, $dim) | rand() ] as vector
    call db.index.vector.queryNodes($index_name, 1, vector) yield node
    return node
    ''',
    index_name = index_name,
    dim = dim,
    routing_= RoutingControl.READ,
    database_= db_name
)
for record in records:
    print(record)

<Record node=<Node element_id='4:4f7d19cf-d031-401b-8a7f-5dcacef86620:310' labels=frozenset({'Node'}) properties={'vector': [0.4357869625091553, 0.7140089273452759, 0.5068435668945312, 0.7121810913085938, 0.6909042000770569, 0.8855467438697815, 0.008132762275636196, 0.4999270737171173, 0.047485943883657455, 0.29731711745262146, 0.0784582868218422, 0.757503867149353, 0.44159165024757385, 0.522188663482666, 0.9904033541679382, 0.7339659929275513, 0.605324923992157, 0.0721699669957161, 0.07483122497797012, 0.5932315587997437, 0.8121064305305481, 0.06005365401506424, 0.4122338593006134, 0.6779442429542542, 0.2790108323097229, 0.5505051612854004, 0.6940599679946899, 0.6302825212478638, 0.6275733709335327, 0.7412080764770508, 0.5704429149627686, 0.9825150370597839, 0.3688940405845642, 0.3819827139377594, 0.38873952627182007, 0.2184697836637497, 0.6065251231193542, 0.4249429702758789, 0.6806058287620544, 0.39480307698249817, 0.6519923806190491, 0.9057116508483887, 0.9821673631668091, 0.186648